<a href="https://colab.research.google.com/github/cnndabbler/agentic-workflows/blob/main/run_pydantic_ai_with_Ollama_and_DeepSeek_R1_Step_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pydantic Agents running DeepSeek-R1 using Ollama

In [1]:
!pip install -q pydantic_ai
!pip install -q pydantic
!pip install -q rich

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.2/223.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 67.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.38.0 which is incompatible.


In [2]:
import subprocess

# Run the Ollama installation script
subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True, check=True)

# Start the Ollama server in the background
ollama_process = subprocess.Popen(["ollama", "serve"])

# Reasoner model
!ollama pull deepseek-r1:32b > /dev/null 2>&1
# Parser and Judge models
!ollama pull qwen2.5 > /dev/null 2>&1

!ollama list

NAME               ID              SIZE      MODIFIED               
qwen2.5:latest     845dbda0ea48    4.7 GB    Less than a second ago    
deepseek-r1:32b    38056bbcbb2d    19 GB     33 seconds ago            


# Ollama Models

In [4]:
from typing import List
from pydantic import BaseModel, Field, confloat
import asyncio
from pydantic_ai import Agent, RunContext
from rich.console import Console
from rich.table import Table
from rich import box

In [5]:
# Initialize models
# reasoner_model = "ollama:deepseek-r1:14b"  # Main reasoning model
# formatter_model = "ollama:qwen2.5"  # Formatter model

from pydantic_ai.models.openai import OpenAIModel

# DeepSeekR1
deepseek_reasoner_model = OpenAIModel(
    'deepseek-r1:32b',
    base_url="http://localhost:11434/v1",
    api_key="whatever",
)

deepseek_parser_model = OpenAIModel(
    'qwen2.5',
    base_url="http://localhost:11434/v1",
    api_key="whatever",
)

# Pydantic Classes

In [6]:
from typing import List
from pydantic import BaseModel, Field, confloat
import asyncio
from pydantic_ai import Agent, RunContext
from rich.console import Console
from rich.table import Table
from rich import box

# Define the Joke model
class Joke(BaseModel):
    setup: str = Field(..., description="The setup part of the joke")
    punchline: str = Field(..., description="The punchline of the joke")
    category: str = Field(..., description="Category of the joke (e.g., 'pun', 'wordplay', 'dad joke')")
    thinking: str = Field(..., description="the thought process and planning")

class JokeAssessment(BaseModel):
    originality: confloat(ge=0, le=10) = Field(..., description="Rating of joke originality from 0-10")
    humor_level: confloat(ge=0, le=10) = Field(..., description="Rating of humor level from 0-10")
    family_friendly: bool = Field(..., description="Whether the joke is family-friendly")
    wordplay_quality: confloat(ge=0, le=10) = Field(..., description="Rating of wordplay quality from 0-10")
    delivery_structure: confloat(ge=0, le=10) = Field(..., description="Rating of setup-punchline structure from 0-10")
    strengths: List[str] = Field(..., description="List of joke's strong points")
    improvement_suggestions: List[str] = Field(..., description="Think step buy step and provide a very extensive List of very specific suggestions for improvement")
    overall_score: confloat(ge=0, le=10) = Field(..., description="Overall joke quality score from 0-10")


In [7]:
# Create Joke Creator Agent
joke_creator = Agent(
    model=deepseek_reasoner_model,
    result_type=str,
    deps_type=str,  # Takes a string prompt as input
    system_prompt=(
        "You are a joke creator that specializes in creating funny, family-friendly dad jokes. "
        "Your output MUST follow this EXACT format with EXACT field names:\n\n"
        "Thinking: \n<think>\n[your detailed thought process without changes]\n</think>\n\n"
        "Setup: [setup text]\n"
        "Punchline: [punchline text]\n"
        "Category: [category]\n"
        # "Thinking: [brief explanation]\n"
    ),
)

# Pydantic AI with DeepSeek-R1: No class binding

Without specifying the Joke class

In [8]:
joke_text = await joke_creator.run("Create a dad joke")
print("\nRaw joke text:")
print("-" * 50)
print(joke_text.data)
print("-" * 50)


Raw joke text:
--------------------------------------------------
<think>
Alright, so the user wants me to create a dad joke. They've given specific formatting instructions with "Thinking:" and "Create a dad joke". I need to make sure the joke is family-friendly and fits their required structure.

First, considering dad jokes are usually puns or plays on words. Let's brainstorm some everyday objects that can have double meanings. Maybe something related to school since it's relatable for families.

"Note" comes to mind because it has multiple meanings – a written note, like in music or school assignments, and the idea of giving a kid notes they need to learn. It can also tie into a teacher’s perspective, adding a layered joke.

So, using "note" as the core. The setup needs to engage and lead to the punchline smoothly. Maybe asking how much a teacher values their students? Then the punchline reveals it's by the number of notes they give, which is both about music notes and maybe detent

# Binding to Pydantic Joke class: Failure

- You will see the following error:
- BadRequestError: Error code: 400 - {'error': {'message': 'registry.ollama.ai/library/deepseek-r1:32b does not support tools', 'type': 'api_error', 'param': None, 'code': None}}

In [9]:
# Create Joke Creator Agent
joke_creator_with_pydantic_class = Agent(
    model=deepseek_reasoner_model,
    result_type=Joke,
    deps_type=str,  # Takes a string prompt as input
    system_prompt=(
        "You are a joke creator that specializes in creating funny, family-friendly dad jokes. "
        "Your output MUST follow this EXACT format with EXACT field names:\n\n"
        "Thinking: \n<think>\n[your detailed thought process without changes]\n</think>\n\n"
        "Setup: [setup text]\n"
        "Punchline: [punchline text]\n"
        "Category: [category]\n"
        # "Thinking: [brief explanation]\n"
    ),
)

In [10]:
joke_text = await joke_creator_with_pydantic_class.run("Create a dad joke")
print("\nRaw joke text:")
print("-" * 50)
print(joke_text.data)
print("-" * 50)

BadRequestError: Error code: 400 - {'error': {'message': 'registry.ollama.ai/library/deepseek-r1:32b does not support tools', 'type': 'api_error', 'param': None, 'code': None}}

# Using a Parser Agent to successfully bind to Pydantic Class

In [11]:
# Create Parser Agent
joke_parser = Agent(
    model=deepseek_parser_model,
    result_type=Joke,
    deps_type=str,  # Takes raw text as input
    system_prompt=(
        "You are a joke parser that extracts components from the input text to create a Joke object. "
        "Your ONLY task is to find and extract these components from the input text:\n"
        "1. The setup (after 'Setup:')\n"
        "2. The punchline (after 'Punchline:')\n"
        "3. The category (after 'Category:')\n"
        "4. The thinking (between <think> tags)\n\n"
        "Return the components in this format:\n"
        "setup=[text after Setup:]\n"
        "punchline=[text after Punchline:]\n"
        "category=[text after Category:]\n"
        "thinking=[text between <think> tags]\n\n"
        "DO NOT generate new content or modify the text in any way.\n"
        "DO NOT include any other text in your response."
    ),
)



In [24]:
# Step 1: Create raw joke text using DeepSeek-R1
print("Creating joke...")
joke_text = await joke_creator.run("Create a dad joke")
if not joke_text:
    raise ValueError("Failed to create joke")
print("\nRaw joke text:")
print("-" * 50)
print(joke_text.data)
print("-" * 50)

Creating joke...

Raw joke text:
--------------------------------------------------
<think>
Alright, the user wants me to create a family-friendly dad joke. They specified that the output must follow an exact format with specific field names, including Thinking and a structured response. 

First, I need to think of a topic that's universally relatable but also allows for a pun. Dinosaurs are always a safe bet—they’re fun and not too adult-oriented.

Next, I'll brainstorm what dinosaurs might do around the house or during free time. Maybe something they could "dig" into, which is both literal and a play on words. 

For the punchline, it should tie back to the setup in a surprising way that makes people chuckle without being offensive. Using "rocking the Casbah" adds a humorous twist because it's unexpected and ties into the idea of dinosaurs exploring different areas.

Finally, I need to ensure the joke fits the family-friendly requirement. It’s light-hearted, doesn’t involve any adult 

In [12]:
# Step 2: Parse the text into a Pydantic model using Qwen2.5
print("\nParsing joke...")
parsed_joke = await joke_parser.run(joke_text.data)
if not parsed_joke:
    raise ValueError("Failed to parse joke")
print("\nParsed joke text:")
print("-" * 50)
print(parsed_joke.data)
print("-" * 50)



Parsing joke...

Parsed joke text:
--------------------------------------------------
setup='Why did the teacher value her students?' punchline='Because she graded them by the number of *notes* they played!' category='School/humor' thinking='Thinking: Let me come up with a Dad joke about something relatable but funny. Notes are used in music and school, which is perfect for a play on words. Teachers give notes to students or play notes on instruments. It’s simple and family-friendly.'
--------------------------------------------------


In [13]:
# Step 3: Display results
console = Console()

# Create assessment table
table = Table(title="Joke Parsed", box=box.ROUNDED)
table.add_column("Property", style="cyan", width=20)
table.add_column("Value", style="yellow")

# Add joke content
table.add_row("Setup", parsed_joke.data.setup)
table.add_row("Punchline", parsed_joke.data.punchline)
table.add_row("Category", parsed_joke.data.category)
table.add_row("","")


# Add thinking process
table.add_row("", "")
table.add_row("[bold]Thinking Process", "")
thinking_lines = parsed_joke.data.thinking.split('\n')
for line in thinking_lines:
    if line.strip():
        table.add_row("", line)

# Print the table
console.print("\n")
console.print(table)

                                                    Joke Parsed                                                    
╭──────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────╮
│ Property             │ Value                                                                                    │
├──────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────┤
│ Setup                │ Why did the teacher value her students?                                                  │
│ Punchline            │ Because she graded them by the number of *notes* they played!                            │
│ Category             │ School/humor                                                                             │
│                      │                                                                                          │
│                      │                                                                                          │
│ Thinking Process     │                                                                                          │
│                      │ Thinking: Let me come up with a Dad joke about something relatable but funny. Notes are  │
│                      │ used in music and school, which is perfect for a play on words. Teachers give notes to   │
│                      │ students or play notes on instruments. It’s simple and family-friendly.                  │
╰──────────────────────┴──────────────────────────────────────────────────────────────────────────────────────────╯